# Notebook for preprocessing Wikipedia (English) dataset

### Initilizing phonemizer and tokenizer

In [1]:
import yaml

config_path = "../PL-BERT/Configs/config.yml" # you can change it to anything else
config = yaml.safe_load(open(config_path))

In [2]:
import re

# This matches any character NOT in:
# - Basic Latin (\u0000-\u007F)
# - Latin-1 Supplement (\u00A0-\u00FF)
# and also ignores common punctuation and numbers (which are in \u0000-\u007F anyway)
non_latin_pattern = re.compile(r'[^\u0000-\u007F\u00A0-\u00FF\u0100-\u024F]')

def contains_non_latin(word):
    return bool(non_latin_pattern.search(word))


In [56]:
#from phonemize import phonemize
import string

def phonemize(text, global_phonemizer, vocabulary):
    words = text.split()
    
    phonemes_bad = [global_phonemizer.phonemize([word.replace("@@","")], strip=True)[0] if word not in string.punctuation else word for word in words]
    input_ids = []
    phonemes = []
    
    for i in range(len(words)):
        word = words[i]
        phoneme = phonemes_bad[i]

        if contains_non_latin(word):
            continue

        input_ids.append(vocabulary[word])
        phonemes.append(phoneme)
      

    assert len(input_ids) == len(phonemes)
    return {'input_ids' : input_ids, 'phonemes': phonemes}

In [16]:
import phonemizer
global_phonemizer = phonemizer.backend.EspeakBackend(language='cs', preserve_punctuation=True,  with_stress=True)

In [4]:
#from phonemizer import phonemize
#from phonemizer.backend import EspeakBackend
#backend = EspeakBackend('cs')
#global_phonemizer.phonemize([t["text"]])

In [5]:
#t["text"]

In [6]:
#tokenizer.tokenize(t["text"])

In [4]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(config['dataset_params']['tokenizer']) # you can use any other tokenizers if you want to

### Process dataset

In [77]:
from datasets import load_dataset
#dataset = load_dataset("wikimedia/wikipedia", "20231101.cs")['train'] # you can use other version of this dataset
dataset = load_dataset("text", data_files="cswiki_bpe.txt")
dataset = dataset['train']


Generating train split: 0 examples [00:00, ? examples/s]

In [78]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 13516925
    })
})

In [91]:
from datasets import load_dataset
#dataset = load_dataset("wikimedia/wikipedia", "20231101.cs")['train'] # you can use other version of this dataset
dataset = load_dataset("text", data_files="newstest2013_bpe.cs")
dataset = dataset['train']

Generating train split: 0 examples [00:00, ? examples/s]

In [41]:
def chunk_file(filepath, min_tokens=512):
    buffer = []
    with open(filepath, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            # add line + space separator
            if buffer:
                buffer.extend(line.split())
            else:
                buffer = line.split()

            if len(buffer) >= min_tokens:
                yield {"text": " ".join(buffer[:512])}
                buffer = buffer[512:]
    # flush remainder
    if buffer:
        yield {"text": " ".join(buffer)}

from datasets import Dataset

dataset = Dataset.from_generator(lambda: chunk_file("cswiki_bpe.txt", min_tokens=512))


Generating train split: 0 examples [00:00, ? examples/s]

In [48]:
vocabulary = {}

with open("vocab.cs", "r", encoding="utf-8") as f:
    for row_num, line in enumerate(f, start=1):
        word, _ = line.strip().split(maxsplit=1)  # split into word and occurrences
        vocabulary[word] = row_num


In [68]:
token_maps = {} 
for word in vocabulary:
    id = vocabulary[word]
    token_maps[id] = {"word" : word, "token": id}

In [85]:
root_directory = "./wiki_phoneme_bpe" # set up root directory for multiprocessor processing

In [84]:
import os
num_shards = 1000

def process_shard(i):
    directory = root_directory + "/shard_" + str(i)
    if os.path.exists(directory):
        print("Shard %d already exists!" % i)
        return
    print('Processing shard %d ...' % i)
    shard = dataset.shard(num_shards=num_shards, index=i)
    processed_dataset = shard.map(lambda t: phonemize(t['text'], global_phonemizer, vocabulary))
    if not os.path.exists(directory):
        os.makedirs(directory)
    processed_dataset.save_to_disk(directory)

In [38]:
t = dataset[0]
t["text"]

'německá slova a fráze'

In [75]:

#p = phonemize('Čína (čínsky v českém přepisu Čung-kuo, pchin-jinem Zhōngguó, znaky zjednodušené 中国), plným názvem Čínská lidová republika (čínsky v českém přepisu Čung-chua žen-min kung-che-kuo, pchin-jinem Zhōnghuá rénmín gònghéguó, znaky zjednodušené 中华人民共和国), zkratkou ČLR (mezinárodně CN),[2][3] je stát ležící ve východní Asii. S 1,4 miliardy obyvatel je Čína, těsně po Indii,[4] druhou nejlidnatější zemí světa, rozloha 9,6 milionu km² ji činí čtvrtým největším státem.', global_phonemizer, tokenizer)


In [92]:
process_shard(0)

Processing shard 0 ...


Map:   0%|          | 0/600 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/600 [00:00<?, ? examples/s]

In [86]:
from pebble import ProcessPool
from concurrent.futures import TimeoutError

#### Note: You will need to run the following cell multiple times to process all shards because some will fail. Depending on how fast you process each shard, you will need to change the timeout to a longer value to make more shards processed before being killed.


In [93]:
max_workers = 20 # change this to the number of CPU cores your machine has 

with ProcessPool(max_workers=max_workers) as pool:
    pool.map(process_shard, range(num_shards), timeout=180)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Shard 0 already exists!Processing shard 1 ...Processing shard 4 ...Processing shard 2 ...Processing shard 3 ...






Map:   0%|          | 0/599 [00:00<?, ? examples/s]

Map:   0%|          | 0/600 [00:00<?, ? examples/s]

Map:   0%|          | 0/600 [00:00<?, ? examples/s]

Map:   0%|          | 0/599 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/600 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/600 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/599 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/599 [00:00<?, ? examples/s]

### Collect all shards to form the processed dataset

In [94]:
from datasets import load_from_disk, concatenate_datasets

output = [dI for dI in os.listdir(root_directory) if os.path.isdir(os.path.join(root_directory,dI))]
datasets = []
for o in output:
    directory = root_directory + "/" + o
    try:
        shard = load_from_disk(directory)
        datasets.append(shard)
        print("%s loaded" % o)
    except:
        continue

shard_2 loaded
shard_4 loaded
shard_1 loaded
shard_3 loaded
shard_0 loaded


In [62]:
dataset = concatenate_datasets(datasets)
dataset.save_to_disk(config['data_folder'])
print('Dataset saved to %s' % config['data_folder'])

Saving the dataset (0/13 shards):   0%|          | 0/557495 [00:00<?, ? examples/s]

Dataset saved to wikipedia_20231101.cs.processed


In [95]:
dataset = concatenate_datasets(datasets)
dataset.save_to_disk("newstest2013.cs.processed")
print('Dataset saved to %s' % "newstest2013.cs.processed")

Saving the dataset (0/1 shards):   0%|          | 0/2998 [00:00<?, ? examples/s]

Dataset saved to newstest2013.data.cs


In [96]:
# check the dataset size
dataset

Dataset({
    features: ['text', 'input_ids', 'phonemes'],
    num_rows: 2998
})

In [ ]:
dataset = datasets.load_dataset(config['data_folder'])

### Remove unneccessary tokens from the pre-trained tokenizer
The pre-trained tokenizer contains a lot of tokens that are not used in our dataset, so we need to remove these tokens. We also want to predict the word in lower cases because cases do not matter that much for TTS. Pruning the tokenizer is much faster than training a new tokenizer from scratch. 

In [17]:
max_workers = 24

In [2]:
from simple_loader import FilePathDataset, build_dataloader

file_data = FilePathDataset(dataset)
loader = build_dataloader(file_data, num_workers=22, batch_size=128)

ModuleNotFoundError: No module named 'simple_loader'

In [19]:
special_token = config['dataset_params']['word_separator']

In [20]:
# get all unique tokens in the entire dataset

from tqdm import tqdm

unique_index = [special_token]
for _, batch in enumerate(tqdm(loader)):
    unique_index.extend(batch)
    unique_index = list(set(unique_index))

  1%|▏                                        | 21/4172 [00:00<01:16, 54.52it/s]

[148, 62403, 16409, 45402, 190, 155, 50555, 11147, 10132, 102877]
[26676, 10162, 17700, 36905, 17339, 45118, 16764, 190, 94755, 190]
[12631, 12546, 16050, 10354, 11806, 41547, 113, 10250, 119, 29760]
[155, 10157, 52043, 10667, 10371, 97764, 45388, 20639, 113, 11590]
[10190, 12713, 19201, 10465, 113, 194, 19610, 17246, 10104, 12713]
[47894, 46680, 10144, 27546, 82560, 117, 76849, 19706, 10112, 18883]
[41512, 11147, 10410, 11886, 10144, 39918, 10132, 194, 85772, 13258]
[49056, 23388, 25039, 93744, 32009, 10123, 27565, 190, 11636, 13330]
[45031, 104270, 10107, 1725, 58240, 13504, 10123, 100, 117, 32348]
[109292, 13434, 25768, 351, 13051, 12951, 113, 115, 10258, 119]
[22177, 36947, 25247, 14854, 16081, 13577, 10123, 10335, 119, 113]
[153, 22165, 10418, 97608, 10281, 106896, 10174, 11573, 10545, 27565]
[14915, 13072, 24829, 113, 33195, 11011, 181, 182, 119, 181]
[17640, 16889, 16764, 10144, 39198, 53922, 117, 31828, 39158, 16050]
[22532, 53696, 16354, 113, 125, 119, 26564, 39343, 117, 53068

  1%|▍                                        | 44/4172 [00:00<00:54, 76.33it/s]

[53068, 10526, 11451, 16986, 18823, 10251, 11398, 10104, 14257, 62014]
[13759, 23603, 20124, 117, 38781, 10147, 62372, 31347, 10457, 75311]
[159, 44703, 77994, 58240, 93682, 10707, 141, 44703, 14678, 87812]
[12727, 113, 32112, 65622, 100886, 12674, 12674, 12674, 114, 11398]
[32352, 85886, 53453, 78432, 10362, 113, 48267, 18939, 18072, 11477]
[17526, 18733, 11595, 10202, 120, 10203, 10144, 10186, 119, 25470]
[63672, 23420, 113, 11005, 42008, 10280, 114, 74768, 67553, 58327]
[138, 15983, 15594, 10269, 12090, 10291, 40442, 10144, 52493, 78656]
[10117, 55845, 36688, 10111, 44557, 131, 138, 24729, 10108, 17432]
[11791, 96970, 10661, 24301, 10144, 39198, 27565, 83426, 10562, 131]
[44795, 10401, 16887, 10113, 17216, 11530, 113, 115, 128, 119]
[71164, 15774, 11169, 10598, 113, 10931, 37199, 76153, 131, 763]
[159, 10157, 17054, 71595, 24441, 10306, 119, 92264, 10151, 13650]
[13124, 46791, 10113, 10144, 10380, 84876, 38703, 13330, 10132, 15069]
[11255, 12617, 87808, 113, 114, 10144, 93434, 49807

  2%|▋                                        | 68/4172 [00:01<00:50, 80.88it/s]

...

[45756, 10219, 21631, 14488, 16409, 113, 123, 119, 32850, 11521]
[28117, 28248, 30509, 34935, 84321, 63488, 10116, 117, 16187, 87333]
[100146, 53082, 150, 119, 141, 119, 16718, 18465, 49917, 11398]
[101798, 10104, 10688, 113, 115, 10273, 119, 32850, 10665, 71671]
[49056, 20986, 32422, 113, 50288, 119, 131, 49056, 20986, 22996]
[145, 25768, 10157, 77838, 16420, 13208, 100, 25369, 10171, 19706]
[22787, 82560, 54952, 67669, 11630, 117, 50302, 10157, 77727, 62372]
[68629, 14074, 13983, 60576, 79689, 10123, 113, 10824, 25849, 21087]
[20932, 10602, 118, 24266, 118, 10104, 118, 23859, 88348, 10129]
[43878, 10545, 29872, 10144, 17183, 20909, 11451, 87530, 10323, 44419]
[33845, 61090, 25063, 10335, 113, 52812, 119, 153, 10305, 11733]
[140, 26179, 10550, 29258, 17352, 28372, 10144, 346, 18246, 12805]
[345, 31112, 68507, 19012, 10758, 56493, 10144, 53508, 57677, 190]
[31826, 106133, 14074, 14868, 10350, 194, 13740, 37661, 34660, 10853]
[12275, 15846, 113, 115, 10413, 119, 33900, 11200, 117, 17111

100%|█████████████████████████████████████▊| 4154/4172 [00:41<00:00, 113.01it/s]

[24825, 21699, 55485, 10419, 10126, 25231, 190, 28170, 163, 29516]
[76594, 91577, 69286, 59206, 190, 20648, 10661, 50276, 79323, 94260]
[10421, 13866, 30462, 113, 115, 10193, 119, 31533, 11063, 103205]
[10984, 73349, 113, 2008, 16463, 15396, 36394, 82554, 117, 12271]
[41432, 10144, 10126, 57217, 46450, 190, 58509, 10339, 61401, 78886]
[15945, 11474, 10545, 29872, 113, 10314, 119, 30387, 11932, 117]
[66033, 21631, 23222, 351, 96666, 10419, 113, 65089, 10147, 43399]
[20977, 14912, 36064, 10116, 49056, 16555, 113, 115, 10208, 119]
[31826, 10161, 32595, 71322, 11859, 59484, 113, 114, 11398, 11859]
[19902, 27899, 113, 115, 10825, 114, 10144, 52676, 15614, 48606]
[144, 30101, 11679, 10144, 10150, 119, 181, 11636, 28036, 15009]
[34817, 25125, 179, 18929, 10144, 102508, 117, 31828, 84125, 27101]
[32701, 30186, 22165, 12354, 113, 122, 119, 30387, 11932, 94042]
[11916, 73099, 75256, 10144, 280, 68498, 13117, 26142, 25186, 187]
[371, 10116, 13208, 18647, 10477, 105386, 189, 27622, 10567, 16757]
[

100%|██████████████████████████████████████| 4172/4172 [00:41<00:00, 100.37it/s]

[17385, 60304, 10269, 113, 69492, 131, 17385, 60304, 10269, 132]
[159, 89878, 10477, 11202, 152, 19516, 36867, 25947, 10144, 17709]
[20932, 10602, 118, 53153, 118, 10110, 118, 20304, 13695, 113]
[15807, 73503, 15490, 117, 32952, 18200, 62372, 15807, 351, 10526]
[14321, 27199, 10115, 10144, 49807, 190, 11061, 117, 10323, 58608]
[20932, 10602, 118, 12944, 43815, 118, 122, 113, 12127, 119]
[26259, 32439, 21333, 23239, 10157, 113, 10386, 119, 29760, 13630]


In [43]:
unique_index[:20]

[0,
 100,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123]

In [69]:
import pickle
with open(config['dataset_params']['token_maps'], 'wb') as handle:
    pickle.dump(token_maps, handle)
print('Token mapper saved to %s' % config['dataset_params']['token_maps'])

Token mapper saved to token_maps.pkl


### Test the dataset with dataloader


In [76]:
from dataloader import build_dataloader

train_loader = build_dataloader(dataset, batch_size=32, num_workers=0, dataset_config=config['dataset_params'])

177


In [71]:
_, (words, labels, phonemes, input_lengths, masked_indices) = next(enumerate(train_loader))

In [72]:
words.shape

torch.Size([32, 512])

In [73]:
phonemes

tensor([[ 29,  29,  29,  ..., 147,  90, 157],
        [ 62,  16, 156,  ..., 156,  57,  58],
        [ 61,  47,  16,  ..., 158,  62,  47],
        ...,
        [ 16,  37,  16,  ..., 156,  57,  53],
        [ 29,  29,  29,  ..., 156,  51,  62],
        [  4,  16, 156,  ...,  47, 102,  16]])

In [74]:
labels

tensor([[ 61,  62,  60,  ..., 147,  90, 157],
        [ 62,  16, 156,  ..., 156,  57,  58],
        [ 61,  47,  16,  ..., 158,  62,  47],
        ...,
        [ 16,  37,  16,  ..., 156,  57,  53],
        [ 62,  61,  47,  ..., 156,  51,  62],
        [  4,  16, 156,  ...,  47, 102,  16]])

In [35]:
dataset[0]

{'id': '260002',
 'url': 'https://cs.wikipedia.org/wiki/Simona%20Sta%C5%A1ov%C3%A1',
 'title': 'Simona Stašová',
 'input_ids': [98350,
  10838,
  10113,
  34050,
  14257,
  113,
  115,
  10270,
  119,
  31554,
  11071,
  17410,
  114,
  10144,
  60099,
  91623,
  119,
  371,
  14865,
  10123,
  68333,
  10113,
  10126,
  10149,
  51570,
  19353,
  30869,
  30822,
  25964,
  10756,
  27622,
  109788,
  82661,
  169,
  17071,
  52814,
  25201,
  139,
  16852,
  13434,
  27408,
  10838,
  10113,
  14747,
  119,
  146,
  10514,
  25470,
  10305,
  53342,
  26378,
  32952,
  84589,
  22278,
  10129,
  106146,
  10116,
  95108,
  14412,
  190,
  32950,
  19275,
  119,
  11255,
  102835,
  19404,
  18840,
  14210,
  50846,
  16852,
  91543,
  10113,
  10132,
  47855,
  11517,
  12022,
  117,
  10514,
  16803,
  78728,
  40423,
  190,
  11409,
  10693,
  57852,
  10113,
  87095,
  16494,
  190,
  79874,
  10269,
  63623,
  11636,
  71056,
  11130,
  18774,
  117,
  26624,
  10247,
  13595,
  1